In [ ]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
from functions import util

import warnings
warnings.filterwarnings("ignore")

# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='player_stats',
    version=1,
)
game_events_fg = fs.get_feature_group(
    name='game_events',
    version=1,
)

# Select features for training data
selected_features = player_stats_fg.select_all().join(game_events_fg.select_all(), on=['player_id'])
selected_features.show(10)

feature_view = fs.get_or_create_feature_view(
    name='clash_royale_fv',
    description="Clash Royale game prediction features",
    version=1,
    labels=['game_outcome'],
    query=selected_features,
)

start_date_test_data = "2024-12-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

# Split data into train and test sets
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)

# Drop unnecessary columns
train_features = X_train.drop(['date', 'player_id'], axis=1)
test_features = X_test.drop(['date', 'player_id'], axis=1)

# Create and train the XGBoost Regressor
xgb_regressor = XGBRegressor()
xgb_regressor.fit(train_features, y_train)

# Predict game outcomes on the test set
y_pred = xgb_regressor.predict(test_features)

# Calculate and print metrics
mse = mean_squared_error(y_test.iloc[:, 0], y_pred)
print("MSE:", mse)

r2 = r2_score(y_test.iloc[:, 0], y_pred)
print("R squared:", r2)

# Save predictions to DataFrame
df = y_test
df['predicted_game_outcome'] = y_pred

# Create directory for model artifacts
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Visualize and save feature importance
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define input and output schemas
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_dict = model_schema.to_dict()

# Save the XGBoost model
xgb_regressor.save_model(model_dir + "/model.json")
mr = project.get_model_registry()

# Register the model in the Hopsworks model registry
cr_model = mr.python.create_model(
    name="clash_royale_xgboost_model",
    metrics={"mse": mse, "r2": r2},
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="Clash Royale game outcome predictor",
)

# Save the model artifacts to the registry
cr_model.save(model_dir)
